# Small Oscillations Double Pendulum

##Preamble

In [1]:
import sympy as sp

Symbolic variables

In [2]:
t,l1,l2,m1,m2,g=sp.symbols('t,l1,l2,m1,m2,g',positive=True)

Generalized coordinates

In [3]:
phi1,phi2=sp.Function('phi1')(t),sp.Function('phi2')(t)

Cartesian Coordinates

In [4]:
x1=l1*sp.sin(phi1)
x2=l1*sp.sin(phi1)+l2*sp.sin(phi2)
y1=l1*sp.cos(phi1)
y2=l1*sp.cos(phi1)+l2*sp.cos(phi2)

Cartesian velocities

In [5]:
dot_x1=x1.diff(t)
dot_y1=y1.diff(t)
dot_x2=x2.diff(t)
dot_y2=y2.diff(t)

## Kinetic Energy $T$

In [6]:
T=sp.simplify(sp.Rational(1,2)*m1*(dot_x1**2+ dot_y1**2)+sp.Rational(1,2)*m2*(dot_x2**2+ dot_y2**2))
T

l1**2*m1*Derivative(phi1(t), t)**2/2 + m2*(l1**2*Derivative(phi1(t), t)**2 + 2*l1*l2*cos(phi1(t) - phi2(t))*Derivative(phi1(t), t)*Derivative(phi2(t), t) + l2**2*Derivative(phi2(t), t)**2)/2

## Potential Energy $U$ (notice that $y$ increases downwards)

In [7]:
U=(-m1*g*y1-m2*g*y2).simplify()
U

-g*(l1*m1*cos(phi1(t)) + l1*m2*cos(phi1(t)) + l2*m2*cos(phi2(t)))

## Lagrangian $L=T-U$

In [8]:
Lag=(T-U).simplify()
Lag

g*(l1*m1*cos(phi1(t)) + l1*m2*cos(phi1(t)) + l2*m2*cos(phi2(t))) + l1**2*m1*Derivative(phi1(t), t)**2/2 + m2*(l1**2*Derivative(phi1(t), t)**2 + 2*l1*l2*cos(phi1(t) - phi2(t))*Derivative(phi1(t), t)*Derivative(phi2(t), t) + l2**2*Derivative(phi2(t), t)**2)/2

## For simplicity we take $l_1=l_2=l$, $m_1=m_2=m$

In [9]:
l,m=sp.symbols('l,m', positive=True)

## Inertia Tensor

In [10]:
dot_phi=(phi1.diff(t),phi2.diff(t))
M0=sp.zeros(len(dot_phi))
for i in range(len(dot_phi)):
  for j in range(len(dot_phi)):
    M0[i,j]=sp.simplify((T.diff(dot_phi[i])).diff(dot_phi[j])).subs([(phi1,0),(phi2,0)])
M0

Matrix([
[l1**2*(m1 + m2), l1*l2*m2],
[       l1*l2*m2, l2**2*m2]])

In [11]:
M=M0.subs([(l1,l),(l2,l),(m1,m),(m2,m)])
M

Matrix([
[2*l**2*m, l**2*m],
[  l**2*m, l**2*m]])

## Harmonic Tensor

In [12]:
phi=(phi1,phi2)
K0=sp.zeros(len(phi))
for i in range(len(phi)):
  for j in range(len(phi)):
    K0[i,j]=sp.simplify((U.diff(phi[i])).diff(phi[j])).subs([(phi1,0),(phi2,0)])
K0

Matrix([
[g*l1*(m1 + m2),       0],
[             0, g*l2*m2]])

In [13]:
K=K0.subs([(l1,l),(l2,l),(m1,m),(m2,m)])
K

Matrix([
[2*g*l*m,     0],
[      0, g*l*m]])

In this case, the harmonic tensor is already diagonal. Thus we can use it to perform the scale transformation that reshapes it into the unit matrix, and apply the same transformation to the inertia tensor.

In [14]:
MK=sp.simplify(sp.sqrt(K)**(-1)*M*sp.sqrt(K)**(-1))
MK

Matrix([
[            l/g, sqrt(2)*l/(2*g)],
[sqrt(2)*l/(2*g),             l/g]])

Diagonalization $T^{-1} M_K T=M_{KD}$

In [15]:
T,MKD= MK.diagonalize()

Orthonormalization

In [16]:
OMK=sp.simplify(T*(sp.sqrt((sp.Transpose(T)*T))**(-1)))
OMK

Matrix([
[-sqrt(2)/2, sqrt(2)/2],
[ sqrt(2)/2, sqrt(2)/2]])

Verification of orthonormality and diagonalization

In [17]:
sp.simplify(sp.Transpose(OMK)*OMK)

Matrix([
[1, 0],
[0, 1]])

In [18]:
sp.simplify(sp.Transpose(OMK)*MK*OMK-MKD)

Matrix([
[0, 0],
[0, 0]])

Finally, the diagonal matrix of squared eigenfrequencies is the inverse of $M_{KD}$

In [19]:
WD=(MKD)**(-1)
WD

Matrix([
[g/(l*(1 - sqrt(2)/2)),                     0],
[                    0, g/(l*(sqrt(2)/2 + 1))]])

Modal Matrix

In [20]:
S=sp.simplify(sp.sqrt(K)**(-1)*OMK*sp.sqrt(MKD)**(-1))
S

Matrix([
[-sqrt(2)/(2*l*sqrt(m)*sqrt(2 - sqrt(2))), sqrt(2)/(2*l*sqrt(m)*sqrt(sqrt(2) + 2))],
[         1/(l*sqrt(m)*sqrt(2 - sqrt(2))),         1/(l*sqrt(m)*sqrt(sqrt(2) + 2))]])

Normal Coordinates

In [21]:
NC=(S**(-1))*sp.Matrix([[phi1],[phi2]])
NC

Matrix([
[(-sqrt(2)*l*sqrt(m) + l*sqrt(m))*phi1(t)/sqrt(2 - sqrt(2)) + (-sqrt(2)*l*sqrt(m) + 2*l*sqrt(m))*phi2(t)/(2*sqrt(2 - sqrt(2)))],
[                                        sqrt(2)*l*sqrt(m)*sqrt(sqrt(2) + 2)*phi1(t)/2 + l*sqrt(m)*sqrt(sqrt(2) + 2)*phi2(t)/2]])

General Solution for the Angles

In [22]:
A1,A2,B1,B2=sp.symbols('A1,A2,B1,B2')

In [23]:
phi_sol=sp.simplify(S*sp.Matrix([[A1*sp.cos(WD[0,0]*t)+B1*sp.sin(WD[0,0]*t)],[A2*sp.cos(WD[1,1]*t)+B2*sp.sin(WD[1,1]*t)]]))
phi_sol

Matrix([
[       (-sqrt(sqrt(2) + 2)*(A1*cos(2*g*t/(l*(2 - sqrt(2)))) + B1*sin(2*g*t/(l*(2 - sqrt(2))))) + sqrt(2 - sqrt(2))*(A2*cos(2*g*t/(l*(sqrt(2) + 2))) + B2*sin(2*g*t/(l*(sqrt(2) + 2)))))/(2*l*sqrt(m))],
[sqrt(2)*(sqrt(sqrt(2) + 2)*(A1*cos(2*g*t/(l*(2 - sqrt(2)))) + B1*sin(2*g*t/(l*(2 - sqrt(2))))) + sqrt(2 - sqrt(2))*(A2*cos(2*g*t/(l*(sqrt(2) + 2))) + B2*sin(2*g*t/(l*(sqrt(2) + 2)))))/(2*l*sqrt(m))]])

In [29]:
phi1_0,phi2_0,dot_phi1_0, dot_phi2_0=sp.symbols(r'\phi_1^0,\phi_2^0,\dot{\phi}_1^0, \dot{\phi}_2^0')

In [30]:
sols_A=sp.solve([sp.Eq(phi_sol[0].subs(t,0),phi1_0),sp.Eq(phi_sol[1].subs(t,0),phi2_0)],[A1,A2])
sols_A

{A1: -\phi_1^0*l*sqrt(m)/sqrt(sqrt(2) + 2) + sqrt(2)*\phi_2^0*l*sqrt(m)/(2*sqrt(sqrt(2) + 2)),
 A2: \phi_1^0*l*sqrt(m)/sqrt(2 - sqrt(2)) + sqrt(2)*\phi_2^0*l*sqrt(m)/(2*sqrt(2 - sqrt(2)))}

In [31]:
dot_phi_sol=phi_sol.diff(t)
dot_phi_sol

Matrix([
[       (-sqrt(sqrt(2) + 2)*(-2*A1*g*sin(2*g*t/(l*(2 - sqrt(2))))/(l*(2 - sqrt(2))) + 2*B1*g*cos(2*g*t/(l*(2 - sqrt(2))))/(l*(2 - sqrt(2)))) + sqrt(2 - sqrt(2))*(-2*A2*g*sin(2*g*t/(l*(sqrt(2) + 2)))/(l*(sqrt(2) + 2)) + 2*B2*g*cos(2*g*t/(l*(sqrt(2) + 2)))/(l*(sqrt(2) + 2))))/(2*l*sqrt(m))],
[sqrt(2)*(sqrt(sqrt(2) + 2)*(-2*A1*g*sin(2*g*t/(l*(2 - sqrt(2))))/(l*(2 - sqrt(2))) + 2*B1*g*cos(2*g*t/(l*(2 - sqrt(2))))/(l*(2 - sqrt(2)))) + sqrt(2 - sqrt(2))*(-2*A2*g*sin(2*g*t/(l*(sqrt(2) + 2)))/(l*(sqrt(2) + 2)) + 2*B2*g*cos(2*g*t/(l*(sqrt(2) + 2)))/(l*(sqrt(2) + 2))))/(2*l*sqrt(m))]])

In [32]:
sols_B=sp.solve([sp.Eq(dot_phi_sol[0].subs(t,0),dot_phi1_0),sp.Eq(dot_phi_sol[1].subs(t,0),dot_phi2_0)],[B1,B2])
sols_B

{B1: -2*\dot{\phi}_1^0*l**2*sqrt(m)/(2*sqrt(2)*g*sqrt(sqrt(2) + 2) + 4*g*sqrt(sqrt(2) + 2)) + sqrt(2)*\dot{\phi}_2^0*l**2*sqrt(m)/(2*sqrt(2)*g*sqrt(sqrt(2) + 2) + 4*g*sqrt(sqrt(2) + 2)),
 B2: -2*\dot{\phi}_1^0*l**2*sqrt(m)/(-4*g*sqrt(2 - sqrt(2)) + 2*sqrt(2)*g*sqrt(2 - sqrt(2))) - sqrt(2)*\dot{\phi}_2^0*l**2*sqrt(m)/(-4*g*sqrt(2 - sqrt(2)) + 2*sqrt(2)*g*sqrt(2 - sqrt(2)))}

In [33]:
phi_icn=phi_sol.subs([(A1,sols_A[A1]),(A2,sols_A[A2]),(B1,sols_B[B1]),(B2,sols_B[B2])]).simplify()
phi_icn

Matrix([
[        ((sqrt(2) + 2)*(g*(-2 + sqrt(2))*(2*\phi_1^0 + sqrt(2)*\phi_2^0)*cos(2*g*t/(l*(sqrt(2) + 2))) - l*(2*\dot{\phi}_1^0 + sqrt(2)*\dot{\phi}_2^0)*sin(2*g*t/(l*(sqrt(2) + 2)))) + (-2 + sqrt(2))*(g*(sqrt(2) + 2)*(2*\phi_1^0 - sqrt(2)*\phi_2^0)*cos(2*g*t/(l*(2 - sqrt(2)))) + l*(2*\dot{\phi}_1^0 - sqrt(2)*\dot{\phi}_2^0)*sin(2*g*t/(l*(2 - sqrt(2))))))/(4*g*(-2 + sqrt(2))*(sqrt(2) + 2))],
[sqrt(2)*((sqrt(2) + 2)*(g*(-2 + sqrt(2))*(2*\phi_1^0 + sqrt(2)*\phi_2^0)*cos(2*g*t/(l*(sqrt(2) + 2))) - l*(2*\dot{\phi}_1^0 + sqrt(2)*\dot{\phi}_2^0)*sin(2*g*t/(l*(sqrt(2) + 2)))) - (-2 + sqrt(2))*(g*(sqrt(2) + 2)*(2*\phi_1^0 - sqrt(2)*\phi_2^0)*cos(2*g*t/(l*(2 - sqrt(2)))) + l*(2*\dot{\phi}_1^0 - sqrt(2)*\dot{\phi}_2^0)*sin(2*g*t/(l*(2 - sqrt(2))))))/(4*g*(-2 + sqrt(2))*(sqrt(2) + 2))]])